In [114]:
import re
import csv
import tqdm
import pandas as pd

import time
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver as wbd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [115]:
options = wbd.ChromeOptions()
options.add_argument("headless")
options.add_argument("disable-gpu")
driver = wbd.Chrome("./chromedriver.exe", options=options)
# driver = wbd.Chrome(service=Service(ChromeDriverManager().install()))
html = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EB%84%A4%EC%9D%B4%EB%B2%84%EB%8B%AC%EB%A0%A5&oquery=%EB%84%A4%E3%85%A3%E3%85%87%EB%B2%84%EB%8B%AC%EB%A0%A5&tqi=i4EPmlprvmZsslbDSkdssssssU0-356245'
driver.get(html)

C:\Users\jjgus\AppData\Local\Temp\ipykernel_18604\1165236964.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wbd.Chrome("./chromedriver.exe", options=options)


In [125]:
class BussinessDayClassifer():
    
    def __init__(self, end_date, start_date, diff_month = None,
                weekend_list = None, holiday_list = None):
        
        self.end_date = end_date
        self.start_date = start_date
        self.weekend_list = []
        self.holiday_list = []
        self.diff_month = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month
    
    @staticmethod
    def preprocess(list):
        return [datetime.datetime.strptime(i.strftime("%y-%m-%d"), '%y-%m-%d') for i in list] 
    

    def weekend_date(self):
        date_list = pd.date_range(start=self.start_date, end=self.end_date, freq='D')
        for i in date_list:
            if i.weekday() > 4:
                self.weekend_list.append(i)
    
    
    def naver_holiday_date(self):
        start = time.time()  # 시작 시간 저장
        for _ in tqdm.tqdm(range(self.diff_month)): # argument로 개월 수 입력
            html = BeautifulSoup(driver.page_source, 'lxml')
            txt = html.find_all(class_ = 'cld_day')
            yyyymm = html.find_all('span', class_='_title')[0].get_text()
            
            for i, v in enumerate(zip(driver.find_elements(By.CLASS_NAME, 'cld_day'), txt)):
                tmp_list = []
                color = re.sub(r'[^0-9]', '', v[0].value_of_css_property('color'))
                day = v[1].string
                
                if color == '2312312311' or color == '3636361': # 회색, 검은색은 skip
                    continue 

#                 print(f'''idx: {i}, color: {color}, day: {day}''')
                year = int(yyyymm[:4])
                month = int(yyyymm[5:7].lstrip("0"))
                day = int(day.lstrip("0"))

                tmp_list.append(day)
                self.holiday_list.append(datetime.datetime(year, month, day))

            # 한달 전으로 이동
            driver.find_element(By.XPATH, value = '//*[@id="direct_calendar"]/div/div[1]/div[1]/a[2]').click()
            time.sleep(1)

        print("\n[elapsed time]:", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
        
        

In [126]:
end_date= datetime.datetime(2023,6,30) # 마지막 날짜
start_date = datetime.datetime(2023,1,1) # 시작 날짜

In [127]:
BussinessDayClassifer = BussinessDayClassifer(end_date, start_date)
BussinessDayClassifer.diff_month

5

In [123]:
BussinessDayClassifer.naver_holiday_date()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.87s/it]


[elapsed time]: 9.34509825706482


In [128]:
BussinessDayClassifer.holiday_list

[]

In [130]:
BussinessDayClassifer.weekend_date()

In [131]:
BussinessDayClassifer.weekend_list

[Timestamp('2023-01-01 00:00:00', freq='D'),
 Timestamp('2023-01-07 00:00:00', freq='D'),
 Timestamp('2023-01-08 00:00:00', freq='D'),
 Timestamp('2023-01-14 00:00:00', freq='D'),
 Timestamp('2023-01-15 00:00:00', freq='D'),
 Timestamp('2023-01-21 00:00:00', freq='D'),
 Timestamp('2023-01-22 00:00:00', freq='D'),
 Timestamp('2023-01-28 00:00:00', freq='D'),
 Timestamp('2023-01-29 00:00:00', freq='D'),
 Timestamp('2023-02-04 00:00:00', freq='D'),
 Timestamp('2023-02-05 00:00:00', freq='D'),
 Timestamp('2023-02-11 00:00:00', freq='D'),
 Timestamp('2023-02-12 00:00:00', freq='D'),
 Timestamp('2023-02-18 00:00:00', freq='D'),
 Timestamp('2023-02-19 00:00:00', freq='D'),
 Timestamp('2023-02-25 00:00:00', freq='D'),
 Timestamp('2023-02-26 00:00:00', freq='D'),
 Timestamp('2023-03-04 00:00:00', freq='D'),
 Timestamp('2023-03-05 00:00:00', freq='D'),
 Timestamp('2023-03-11 00:00:00', freq='D'),
 Timestamp('2023-03-12 00:00:00', freq='D'),
 Timestamp('2023-03-18 00:00:00', freq='D'),
 Timestamp

In [ ]:
BussinessDayClassifer.diff_month()

In [ ]:
BussinessDayClassifer.holiday_list

In [62]:
holiday_list

[datetime.datetime(2021, 9, 5, 0, 0),
 datetime.datetime(2021, 9, 12, 0, 0),
 datetime.datetime(2021, 9, 19, 0, 0),
 datetime.datetime(2021, 9, 20, 0, 0),
 datetime.datetime(2021, 9, 21, 0, 0),
 datetime.datetime(2021, 9, 22, 0, 0),
 datetime.datetime(2021, 9, 26, 0, 0),
 datetime.datetime(2021, 8, 1, 0, 0),
 datetime.datetime(2021, 8, 8, 0, 0),
 datetime.datetime(2021, 8, 15, 0, 0),
 datetime.datetime(2021, 8, 16, 0, 0),
 datetime.datetime(2021, 8, 22, 0, 0),
 datetime.datetime(2021, 8, 29, 0, 0),
 datetime.datetime(2021, 7, 4, 0, 0),
 datetime.datetime(2021, 7, 11, 0, 0),
 datetime.datetime(2021, 7, 18, 0, 0),
 datetime.datetime(2021, 7, 25, 0, 0),
 datetime.datetime(2021, 6, 6, 0, 0),
 datetime.datetime(2021, 6, 13, 0, 0),
 datetime.datetime(2021, 6, 20, 0, 0),
 datetime.datetime(2021, 6, 27, 0, 0),
 datetime.datetime(2021, 5, 2, 0, 0),
 datetime.datetime(2021, 5, 5, 0, 0),
 datetime.datetime(2021, 5, 9, 0, 0),
 datetime.datetime(2021, 5, 16, 0, 0),
 datetime.datetime(2021, 5, 19, 0

In [67]:
class Oper:
    
    def __init__(self, list = None):
        self.res = 0
        self.list = []
    
    def plus(self, num):
        self.res += num
        self.list.append(self.res)
        return self.res

student1 = Oper()
student2 = Oper()

print(student1.plus(3)) # 3
print(student2.plus(4)) # 4
print(student1.plus(5)) # 8
print(student2.plus(5)) # 9

3
4
8
9


In [68]:
student1.list

[3, 8]

In [69]:
student2.list

[4, 9]